In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import requests


warnings.filterwarnings("ignore")
sns.set(style="whitegrid")

def load_data(db_name="vacancies.db"):
    conn = sqlite3.connect(db_name)
    query = "SELECT * FROM vacancies"
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

data = load_data()

In [2]:
data.head()

,id,name,area_name,salary_from,salary_to,salary_currency,published_at,employer_name,alternate_url,snippet_requirement,snippet_responsibility,professional_roles,schedule,employment,experience
0,110998715,Аналитик данных,Москва,200000.0,250000.0,RUR,2024-12-27,Генотек,https://hh.ru/vacancy/110998715,Умеешь находить аномалии и отклонения в данных...,Проверить и доработать систему отчётности с кл...,"BI-аналитик, аналитик данных",Удаленная работа,Полная занятость,От 3 до 6 лет
1,107157371,Аналитик данных,Москва,50000.0,50000.0,RUR,2024-12-27,Paper Planes,https://hh.ru/vacancy/107157371,Поиска информации в открытых источниках по зад...,Управление группой аналитиков. Возможность уча...,Бизнес-аналитик,Полный день,Полная занятость,От 1 года до 3 лет
2,111501182,Аналитик данных (BI-аналитик/Финансовый аналит...,Екатеринбург,180000.0,220000.0,RUR,2024-12-27,Селлер Капитал,https://hh.ru/vacancy/111501182,Опыт работы аналитиком данных или финансовым а...,Анализ финансовой отчетности и создание прогно...,"BI-аналитик, аналитик данных",Полный день,Полная занятость,От 3 до 6 лет
3,103177161,Аналитик данных,Владивосток,138000.0,138000.0,RUR,2024-12-27,DNS Головной офис,https://hh.ru/vacancy/103177161,Уверенное владение SQL (регулярное использован...,Написание ad-hoc запросов по бизнес-процессам ...,Аналитик,Полный день,Полная занятость,От 1 года до 3 лет
4,96669417,Аналитик данных,Владивосток,109250.0,109250.0,RUR,2024-12-27,DNS Головной офис,https://hh.ru/vacancy/96669417,Опыт работы с SQL. Аналитический склад ума. Си...,"Сбор, верификация, анализ данных по процессам ...","BI-аналитик, аналитик данных",Полный день,Полная занятость,От 1 года до 3 лет


# Расчет метрик качества данных 

In [6]:
from datetime import datetime
df = pd.DataFrame(data)

# Метрика 1: Полнота данных
completeness = df.isnull().mean() * 100

# Метрика 2: Консистентность данных (проверка salary_from < salary_to)
inconsistent_data = (df['salary_from'] > df['salary_to']).mean() * 100

# Метрика 3: Актуальность данных
df['published_at'] = pd.to_datetime(df['published_at'])
current_date = datetime.now()
df['days_since_published'] = (current_date - df['published_at']).dt.days
average_days = df['days_since_published'].mean()

# Метрика 4: Уникальность данных (по столбцу id)
duplicate_percentage = (df.duplicated(subset=['id']).mean()) * 100

# Метрика 5: Качество текстовых данных (средняя длина текста в snippet_requirement)
df['snippet_requirement_length'] = df['snippet_requirement'].apply(len)
average_length = df['snippet_requirement_length'].mean()

# Собираем все метрики в таблицу
metrics = {
    "Метрика": ["Полнота данных", "Консистентность данных", 
                "Актуальность данных (среднее количество дней)", "Уникальность данных", 
                "Качество текстовых данных (средняя длина текста)"],
    "Значение (%)": [completeness.mean(), 100 - inconsistent_data, average_days, 100 - duplicate_percentage, average_length]
}

metrics_df = pd.DataFrame(metrics)

# Выводим красивую таблицу
from tabulate import tabulate

print(tabulate(metrics_df, headers='keys', tablefmt='grid', showindex=False))

+--------------------------------------------------+----------------+
| Метрика                                          |   Значение (%) |
+==================================================+================+
| Полнота данных                                   |        0       |
+--------------------------------------------------+----------------+
| Консистентность данных                           |      100       |
+--------------------------------------------------+----------------+
| Актуальность данных (среднее количество дней)    |        2.27135 |
+--------------------------------------------------+----------------+
| Уникальность данных                              |      100       |
+--------------------------------------------------+----------------+
| Качество текстовых данных (средняя длина текста) |      148.342   |
+--------------------------------------------------+----------------+
